Goal of this notebook:

Tuning the n parameter for IGAR attribute selection algorithm:

Average number of attributes among the projects: 169.16

Minimum number of attributes among the projects: 84


## Tuning strategy

### Collect accuracy for each n

- Vary the n parameter from 1 to 84 (min attributes)
- For each n:
	- For each project:
		- Execute the IGAR using n
		- Execute the prediction (calculate accuracy and normalized improvement)


### Calculate the number of wins for each n:

- For each project:
	- Check which n achieved the best accuracy

IGAR reference: https://www.cs.waikato.ac.nz/~mhall/HallHolmesTKDE.pdf

In [1]:
import numpy as np
import pandas as pd
from IPython.core.display import display

pd.set_option('display.max_columns', None)
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
import warnings
import classifier_utils
warnings.filterwarnings("ignore", category=UserWarning)
import configs

In [2]:
non_features_columns = ["chunk_id", "line_start", "line_end", "line_separator", "kind_conflict", "url", "project"]
non_features_columns.extend(["project_user", "project_name", "path", "file_name", "sha", "leftsha", "rightsha", "basesha"])

In [3]:
selected_dataset = pd.read_csv("../../data/SELECTED_LABELLED_DATASET.csv")
projects = list(selected_dataset['project'].unique())

In [4]:
rf = RandomForestClassifier(random_state=99, n_jobs=5, n_estimators=100, max_features=0.3, min_samples_leaf=1)

In [9]:
min_n = 1
max_n = 84

In [70]:
import importlib
importlib.reload(classifier_utils)
tuning_results = classifier_utils.IGAR_tuning(rf, projects[:6], non_features_columns, min_n, max_n, True)

In [7]:
tuning_results

,project,n,accuracy,accuracy_selected,orig_attr
0,Ramblurr__Anki-Android,1,0.728355,0.554788,129
1,apache__directory-server,1,0.938631,0.908143,96
2,android__platform_frameworks_base,1,0.813874,0.748824,566
3,freenet__fred,1,0.678846,0.593122,134
4,alexo__wro4j,1,0.587674,0.373409,107
...,...,...,...,...,...
2095,jgralab__jgralab,84,0.867922,0.866266,98
2096,sebastianbenz__Jnario,84,0.999021,0.999021,84
2097,CCI-MIT__XCoLab,84,0.973663,0.974995,99
2098,RealVNC__android-frameworks-base-with-screensh...,84,0.811660,0.800132,373


In [34]:
import importlib
importlib.reload(classifier_utils)
summary = classifier_utils.compute_IGAR_tuning_summary(tuning_results, min_n, max_n)

In [35]:
summary

,n,average_default_accuracy,average_accuracy,improvement,average_ranking,number_wins
0,1,0.807345,0.701809,-0.130719,74.08,1
1,2,0.807345,0.730314,-0.095412,73.98,0
2,3,0.807345,0.736112,-0.088231,74.12,0
3,4,0.807345,0.743390,-0.079216,73.42,0
4,5,0.807345,0.744786,-0.077487,72.14,0
...,...,...,...,...,...,...
79,80,0.807345,0.802029,-0.006585,26.80,0
80,81,0.807345,0.804984,-0.002925,17.26,2
81,82,0.807345,0.803093,-0.005267,22.22,1
82,83,0.807345,0.804021,-0.004117,19.20,0


In [33]:
summary.sort_values(['number_wins', 'average_accuracy', 'average_ranking'], ascending=False)

,n,average_default_accuracy,average_accuracy,improvement,average_ranking,number_wins
80,81,0.807345,0.804984,-0.002925,5.136905,2
83,84,0.807345,0.803588,-0.004653,6.726190,2
76,77,0.807345,0.803557,-0.004692,6.946429,2
64,65,0.807345,0.803345,-0.004954,6.982143,2
74,75,0.807345,0.804131,-0.003981,5.583333,1
...,...,...,...,...,...,...
5,6,0.807345,0.754070,-0.065988,21.267857,0
4,5,0.807345,0.744786,-0.077487,21.470238,0
3,4,0.807345,0.743390,-0.079216,21.851190,0
2,3,0.807345,0.736112,-0.088231,22.059524,0


In [30]:
# tuning_results
wins = {}
ranking = {}
for n in range(min_n, max_n+1):
    wins[n] = 0
    ranking[n] = 0
project_results = tuning_results[tuning_results['project']=='alexo__wro4j'].copy()
project_results['ranking'] = project_results['accuracy_selected'].rank(ascending=False)
project_results
for index, row in project_results.iterrows():
    ranking[row['n']] += row['ranking']
print(ranking)
print()
n_projects = len(tuning_results['project'].unique())
for n, accumulated_rank in ranking.items():
    average = accumulated_rank/1
    ranking[n] = average
print(ranking)

{1: 84.0, 2: 83.0, 3: 82.0, 4: 80.0, 5: 81.0, 6: 79.0, 7: 78.0, 8: 77.0, 9: 71.0, 10: 76.0, 11: 72.0, 12: 73.0, 13: 55.0, 14: 75.0, 15: 67.0, 16: 66.0, 17: 63.0, 18: 57.0, 19: 59.0, 20: 60.0, 21: 65.0, 22: 56.0, 23: 50.0, 24: 58.0, 25: 70.0, 26: 49.0, 27: 62.0, 28: 69.0, 29: 54.0, 30: 68.0, 31: 51.0, 32: 74.0, 33: 52.0, 34: 64.0, 35: 61.0, 36: 34.0, 37: 35.0, 38: 19.0, 39: 48.0, 40: 45.0, 41: 4.0, 42: 53.0, 43: 38.0, 44: 5.0, 45: 28.0, 46: 23.0, 47: 46.0, 48: 30.0, 49: 26.0, 50: 36.0, 51: 24.0, 52: 29.0, 53: 8.0, 54: 13.0, 55: 12.0, 56: 18.0, 57: 21.0, 58: 44.0, 59: 20.0, 60: 17.0, 61: 37.0, 62: 7.0, 63: 43.0, 64: 11.0, 65: 1.0, 66: 6.0, 67: 42.0, 68: 25.0, 69: 41.0, 70: 40.0, 71: 47.0, 72: 14.0, 73: 15.0, 74: 31.0, 75: 2.0, 76: 27.0, 77: 10.0, 78: 39.0, 79: 9.0, 80: 33.0, 81: 16.0, 82: 22.0, 83: 3.0, 84: 32.0}

{1: 84.0, 2: 83.0, 3: 82.0, 4: 80.0, 5: 81.0, 6: 79.0, 7: 78.0, 8: 77.0, 9: 71.0, 10: 76.0, 11: 72.0, 12: 73.0, 13: 55.0, 14: 75.0, 15: 67.0, 16: 66.0, 17: 63.0, 18: 57.0, 19: 